In [27]:
import pandas as pd
import numpy as np
from PIL import Image


In [28]:
reflacx_with_clinical_df = pd.read_csv('reflacx_with_clinical.csv', index_col=0)
reflacx_with_clinical_df.columns

In [30]:
## Pick the columns that we want to feed to the model as clinical data
clinical_cols =  ['age', 'gender', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity']

In [31]:
## Retrieve the image as numpy array.

def load_image_array(image_path):
    return np.asarray(Image.open(image_path))


def plot_image_from_array(image_array):
    im = Image.fromarray(image_array)
    im.show()


In [32]:
image_array = load_image_array(reflacx_with_clinical_df.iloc[0]['image_path'])

In [33]:
# plot_image_from_array(image_array)

In [34]:
clinical_df = reflacx_with_clinical_df[clinical_cols]

In [37]:
reflacx_with_clinical_df.head(5)

,id,dicom_id,subject_id,stay_id,study_id,split,image_path,ViewPosition,image_size_x,image_size_y,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
0,P102R108387,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,train,D:\XAMI-MIMIC\patient_18111516\CXR-JPG\s550322...,AP,2544,3056,...,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P102R379837,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,train,D:\XAMI-MIMIC\patient_18111516\CXR-JPG\s550322...,AP,2544,3056,...,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P102R558314,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,train,D:\XAMI-MIMIC\patient_18111516\CXR-JPG\s550322...,AP,2544,3056,...,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,P102R765317,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,train,D:\XAMI-MIMIC\patient_18111516\CXR-JPG\s550322...,AP,2544,3056,...,True,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,P102R915878,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002,55032240,train,D:\XAMI-MIMIC\patient_18111516\CXR-JPG\s550322...,AP,2544,3056,...,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# "pain" -> 10 level rating system
# "acuity" -> 5 level rating system

# Set them as the float

In [21]:
clinical_numerical_cols = ['age', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity']
clinical_categorical_cols = ['gender']

## transform the gender column
# clinical_df['gender'] = clinical_df['gender'].apply(lambda x: 1 if x=="M" else 0)

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
encoders_map = {}
for col in clinical_categorical_cols:
    le = LabelEncoder()
    clinical_df[col] = le.fit_transform(clinical_df[col])
    encoders_map[col] = le

C:\Users\mike8\AppData\Local\Temp\ipykernel_28172\1515832020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical_df[col] = le.fit_transform(clinical_df[col])


In [11]:
clinical_df = clinical_df[clinical_numerical_cols + clinical_categorical_cols]

In [12]:
clinical_df

,age,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,gender
0,69.0,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0,0
1,69.0,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0,0
2,69.0,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0,0
3,69.0,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0,0
4,69.0,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...
3020,43.0,98.0,101.0,18.0,98.0,132.2,72.3,8.0,3.0,0
3023,31.0,97.5,76.0,16.0,100.0,141.0,66.0,2.0,3.0,1
3027,38.0,97.5,115.0,16.0,99.0,187.0,104.0,8.0,2.0,1
3028,42.0,98.6,108.0,18.0,96.0,180.0,109.0,10.0,3.0,0


In [13]:
## design the Fully Connected layer to process 
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class ClinicalNet(nn.Module):
    def __init__(self, num_output_features, numerical_cols, categorical_cols, embedding_dim_maps, dims= [16], ) -> None:
        super(ClinicalNet, self).__init__()

        fcs = []

        for idx, dim in enumerate(dims):
            if idx == 0:
                fcs.append(nn.Linear(len(numerical_cols) + sum(embedding_dim_maps.values()), dim))

            if idx == len(dims) - 1:
                fcs.append(nn.Linear(dim, num_output_features))

            if idx != 0 and idx == len(dim) - 1:            
                fcs.append(nn.Linear(dims[idx-1], dim))
            
        self.net = nn.Sequential(*fcs)

        self.embs = {}  

        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols

        for col in categorical_cols:
            self.embs[col] = nn.Embedding(len(clinical_df[col].unique()), embedding_dim_maps[col])


    def forward(self, x_df):
        ## perform embedding first.

        emb_out = {}    

        for col in self.categorical_cols:
            emb_out[col] = self.embs[col](torch.tensor(x_df[col].array))


        numerical_input = torch.tensor(np.array(x_df[self.numerical_cols])).float()
        
        concat_input = torch.cat(( numerical_input ,*[ emb_out[col] for col in self.categorical_cols ]), dim=1)
        
        return self.net(concat_input)
    


In [16]:
clinical_net = ClinicalNet( num_output_features=64, dims=[32], numerical_cols=clinical_numerical_cols, categorical_cols= clinical_categorical_cols, embedding_dim_maps={'gender': 64})

In [40]:
clinical_df.loc[[1]]

,age,gender,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
1,69.0,F,98.1,90.0,18.0,99.0,184.0,75.0,13.0,3.0


In [17]:
clinical_output = clinical_net(clinical_df[:1])

In [18]:
import torchvision.transforms as transforms

C:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\mike8\anaconda3\envs\pytorch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [19]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

In [20]:
horizontal_flip = True
transforms_lst = [
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip() if horizontal_flip else None,
                transforms.ToTensor(),
                normalize,
            ]
transform_fn = transforms.Compose([t for t in transforms_lst if t])

In [21]:
imgs = [ Image.open(path).convert("RGB") for path in reflacx_with_clinical_df[:5]['image_path']]

In [22]:
example_transformed_input = transform_fn(imgs[0])

In [23]:
example_transformed_input.shape

torch.Size([3, 224, 224])

In [24]:
## Build the densenet for us to feed the image.
from torch.autograd import Variable
from model.densenet import densenet121


class DenseNet(nn.Module):
    def __init__(self, nclasses):
        super(DenseNet, self).__init__()
        self.model_ft = densenet121(pretrained=False, drop_rate=0.1)
        num_ftrs = self.model_ft.classifier.in_features
        self.model_ft.classifier = nn.Linear(num_ftrs, nclasses)
    def forward(self, x):
        return self.model_ft(x)



In [25]:
d_net = DenseNet(64)

In [26]:
d_net.model_ft

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [27]:
cxr_output = d_net.model_ft(example_transformed_input.unsqueeze(0)) ## we can feed in the dense_net now.

In [28]:
fused_representation =  clinical_output + cxr_output

In [29]:
class DecisionNet(nn.Module):
    def __init__(self, num_input_features, num_output_features, dim ) -> None:
        super(DecisionNet, self).__init__()

            
        self.net = nn.Sequential(
            nn.Linear(num_input_features, dim),
            nn.LayerNorm(dim),
            nn.LeakyReLU(.05, inplace=True),
            nn.Linear(dim,  num_output_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)
    


In [30]:
labels_cols = [
    'Airway wall thickening', 'Atelectasis', 'Consolidation',
    'Enlarged cardiac silhouette', 'Fibrosis', 'Fracture',
    'Groundglass opacity', 'Pneumothorax', 'Pulmonary edema',
    'Quality issue', 'Support devices', 'Wide mediastinum',
    'Abnormal mediastinal contour', 'Acute fracture', 'Enlarged hilum',
    'Hiatal hernia', 'High lung volume / emphysema',
    'Interstitial lung disease', 'Lung nodule or mass',
    'Pleural abnormality'
]

In [36]:
all_disease_cols = [
    'Airway wall thickening', 'Atelectasis', 'Consolidation',
    'Enlarged cardiac silhouette', 'Fibrosis', 'Fracture',
    'Groundglass opacity', 'Pneumothorax', 'Pulmonary edema','Wide mediastinum',
    'Abnormal mediastinal contour', 'Acute fracture', 'Enlarged hilum',
    'Hiatal hernia', 'High lung volume / emphysema',
    'Interstitial lung disease', 'Lung nodule or mass',
    'Pleural abnormality'
]

In [37]:
reflacx_with_clinical_df[labels_cols]

,Airway wall thickening,Atelectasis,Consolidation,Enlarged cardiac silhouette,Fibrosis,Fracture,Groundglass opacity,Pneumothorax,Pulmonary edema,Quality issue,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# tansform the diseases from occurence to boolean lables. we don't do this if we want to predict the bouding boxes.
reflacx_with_clinical_df[all_disease_cols] = reflacx_with_clinical_df[all_disease_cols].gt(0)

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
decision_net = DecisionNet(64, len(labels_cols), 32)

In [43]:
# this is the expected output.
decision_out = decision_net(fused_representation)

In [49]:
## try to get loss here


## get bounding boxes according to the reflacx id
loss_fn = nn.MultiLabelSoftMarginLoss()
loss_fn(decision_out, torch.tensor(np.array(reflacx_with_clinical_df[labels_cols].iloc[0])).long().unsqueeze_(0))



tensor(0.9321, grad_fn=<MeanBackward0>)